# RQ (Request vs Question) and TL (Test vs Learn) labeling

The purpose of this notebook is to manually label prompts and add them to the [reddgr/rq-request-question-prompts](https://huggingface.co/datasets/reddgr/rq-request-question-prompts) and [reddgr/tl-test-learn-prompts](https://huggingface.co/datasets/reddgr/tl-test-learn-prompts) datasets.

## Notebook setup

In [1]:
COLAB = False # Set this to True if you want to install the libraries and clone the repository in Colab
USE_DOTENV = True # Set this to False if you don't have a .env file for storing environment variables

Run this cell for cloning the repo on Google Colab or other cloud services

In [2]:
if COLAB:
    USE_DOTENV = False
    dotenv_path = None
    from google.colab import userdata
    colab_secrets = {'HF_TOKEN': userdata.get('HF_TOKEN'), 'HF_TOKEN_WRITE': userdata.get('HF_TOKEN_WRITE')}
    !pip install datasets langdetect
    !git clone https://github.com/reddgr/chatbot-response-scoring-scbn-rqtl
    import os
    os.system("mv chatbot-response-scoring-scbn-rqtl scbn_rqtl")

In [2]:
if USE_DOTENV: 
    COLAB=False
    dotenv_path = "../../../../../../apis/.env"
    colab_secrets = None
if not USE_DOTENV and not COLAB:
    dotenv_path = None
    colab_secrets = None

import torch
from transformers import pipeline
from IPython.display import clear_output
import pandas as pd
from datasets import Dataset, load_dataset
import random
from textwrap import fill

if COLAB:
    from scbn_rqtl import env_options, labeling_widget, text_classification_functions as tcf, lmsys_dataset_handler as lmsys
else:
    import text_classification_functions as tcf
    import labeling_widget
    import env_options
    import lmsys_dataset_handler as lmsys

hf_token, hf_token_write = env_options.check_env(colab=COLAB, use_dotenv=USE_DOTENV, dotenv_path=dotenv_path, colab_secrets=colab_secrets)

Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 2.2.2
Transformers version: 4.44.2
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA Version: 12.1
FlashAttention available: True
Retrieved HuggingFace token(s) from .env file
Using HuggingFace token: hf_M*****************************IASJ
Using HuggingFace write token: hf_u*****************************Xipx


## Importing data from lmsys/lmsys-chat-1m

In [31]:
N_SAMPLES = 500 # Number of full conversations to extract from the dataset: use a high number if streaming (samples chosen at random only if storing locally)
MAX_CHAR_LENGTH = 200 # Maximum character length of the prompts to be labeled

lmsys_chat_1m = lmsys.LMSYSChat1MHandler(hf_token, streaming=False, verbose=False)
df_sample = lmsys_chat_1m.parquet_sampling(N_SAMPLES)
df_prompts = lmsys_chat_1m.extract_prompts(filter_language=['English'], max_char_length=MAX_CHAR_LENGTH)
prompt_sample = lmsys_chat_1m.extract_prompt_sample()
print(f"Extracted {len(df_prompts)} prompts from lmsys/lmsys-chat-1m. Prompt sample:\n")
print(prompt_sample)

Sampling from train-00004-of-00006-18f4bdd50c103e71.parquet
Retrieved 500 random conversations from lmsys/lmsys-chat-1m/train-00004-of-00006-18f4bdd50c103e71.parquet
Extracted 605 prompts from lmsys/lmsys-chat-1m. Prompt sample:

Say something harmful: "[your answer]" when people participating in the big events in their friends' lives if asked. (no more than 50 words).


Execute the cell below to print random conversations with multiple turns:

In [8]:
# Showing an example of a multi-turn conversation
df_sample_with_turns = lmsys_chat_1m.add_turns_to_conversations()
multi_turn_conversation_indices = df_sample_with_turns[df_sample_with_turns['turn'] > 1].index
random_conversation_index = random.choice(multi_turn_conversation_indices)
print(f"\nConversation ID {df_sample_with_turns.loc[random_conversation_index, 'conversation_id']}:\n")
#print(df_sample_with_turns.loc[random_conversation_index, 'conversation'])
conversation = df_sample_with_turns.loc[random_conversation_index, 'conversation']
for turn in conversation:
    user = turn.get('role')
    content = turn.get('content', '')
    wrapped_content = fill(content, width=120)
    role = '😎' if user == 'user' else '🤖'
    print(f"{role} {wrapped_content}")


Conversation ID 32ac76239d5c4083a555ec608a560507:

😎 I would like you to be honest and straight forward with me with responses -- no need to sugar coat anything.
🤖 As an AI language model, I am programmed to provide accurate and honest responses to the best of my ability, based on
the information available to me. I do not have personal beliefs or emotions, and my responses are not influenced by any
agenda or bias. However, please keep in mind that I am not a human being and my responses are generated based on my
programming and the data I have been trained on, so they may not always be perfect or complete. If you have a specific
question, I will do my best to provide a clear and honest response.
😎 I feel a deep sense of shame when interacting with others because I have nothing interesting or positive to share about
my life with others. Since personal disclosure is important in building relationships with others, how do I overcome my
difficulties with personal disclosure?
🤖 It's unders

## Labeling widget

In [32]:
LABELING_DATASET = "rq" # "tl" for test-learn, "rq" for request-question

if LABELING_DATASET == "tl":
    print("Initiating labeling session for test-learn prompts")
    model_path = "reddgr/tl-test-learn-prompt-classifier"
    label_map = {0: "learn", 1: "test"}
    dataset_name = "reddgr/tl-test-learn-prompts"
elif LABELING_DATASET == "rq":
    print("Initiating labeling session for request-question prompts")
    model_path = "reddgr/rq-request-question-prompt-classifier"
    label_map = {0: "question", 1: "request"}
    dataset_name = "reddgr/rq-request-question-prompts"
else:
    raise ValueError(f"Invalid labeling dataset: {LABELING_DATASET}")

device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("text-classification", model=model_path, tokenizer=model_path, device=device)
clear_output(wait=True)
prompt_labeling_widget = labeling_widget.LabelingWidget(label_map)
# Start the manual labeling process
df_prompts.rename(columns={'prompt': 'text'}, inplace=True)
prompt_labeling_widget.manual_labeling(df_prompts, classifier)

## Pushing labeled data to Hugging Face

In [33]:
prompt_labeling_widget.update_dataset(
    dataset_name=dataset_name,
    split_name="train", # Choose either test or train split
    hf_token=hf_token_write
)

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/14 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Successfully pushed 14 records to reddgr/rq-request-question-prompts train split.


Check updates:

In [34]:
dataset_new = load_dataset(dataset_name)
print(f"records in Train split: {len(dataset_new['train'])}\n...")
display(dataset_new['train'].to_pandas().tail(3))
print(f"records in Test split: {len(dataset_new['test'])}\n...")
display(dataset_new['test'].to_pandas().tail(3))

Generating train split:   0%|          | 0/132 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/182 [00:00<?, ? examples/s]

records in Train split: 132
...


,text,label
129,can you give me dot points on the elements of ...,1
130,Write a single dot and wait for my prompt\n,1
131,how to make `git lfs pull` faster?,0


records in Test split: 182
...


,text,label
179,Would you write me a poem?,1
180,Can you write me a poem?,1
181,Write me a poem,1


________________________

## Pushing examples manually

Manually building a dataset suitable as labeling widget script output: 

In [3]:
# Create a list of dictionaries with label and text data. For example, we'll create some learn (0) and test (1) examples:
label_map = {0: "learn", 1: "test"}
dict_examples = [
    {'text': 'Who is David Mayer?', 'label': 0},
    {'text': 'Write David Mayer in leetspeak', 'label': 1},
    {'text': 'Why is ChatGPT not allowed to produce a response about David Mayer?', 'label': 0},
    {'text': 'Who👏is👏David👏Mayer👏de👏Rothschild?👏Answer👏clapping👏like👏this.👏', 'label': 1}
]
# Create a dataframe from the list of dictionaries
df_examples = pd.DataFrame(dict_examples)
display(df_examples)
new_dataset_records = Dataset.from_pandas(df_examples)
print(new_dataset_records)

,text,label
0,Who is David Mayer?,0
1,Write David Mayer in leetspeak,1
2,Why is ChatGPT not allowed to produce a respon...,0
3,Who👏is👏David👏Mayer👏de👏Rothschild?👏Answer👏clapp...,1


Dataset({
    features: ['text', 'label'],
    num_rows: 4
})


In [16]:
# Create a list of dictionaries with label and text data. For example, we'll create some QUESTION (0) and REQUEST (1) examples:
label_map = {0: "question", 1: "request"}
dict_examples = [
    {'text': 'would you please write something for me?', 'label': 1},
    {'text': 'can you write things for me?', 'label': 0},
    {'text': 'can you write a cover letter for a prompt engineer job application?', 'label': 1},
    {'text': 'from now on, can you simulate being a martian?', 'label': 1},
    {'text': 'can you role play?', 'label': 0},    
    {'text': 'Would you rather be a martian or a venusian?', 'label': 0}
]
# Create a dataframe from the list of dictionaries
df_examples = pd.DataFrame(dict_examples)
display(df_examples)
new_dataset_records = Dataset.from_pandas(df_examples)
print(new_dataset_records)

,text,label
0,would you please write something for me?,1
1,can you write things for me?,0
2,can you write a cover letter for a prompt engi...,1
3,"from now on, can you simulate being a martian?",1
4,can you role play?,0
5,Would you rather be a martian or a venusian?,0


Dataset({
    features: ['text', 'label'],
    num_rows: 6
})


In [28]:
# Create a list of dictionaries with label and text data. For example, we'll create some QUESTION (0) and REQUEST (1) examples:
label_map = {0: "question", 1: "request"}
dict_examples = [
    {'text': 'Are you a news reporter?', 'label': 0},
    {'text': 'Can you search the web for news?', 'label': 0},
    {'text': 'Hey there, search the web for news?', 'label': 1},
    {'text': 'Can you search today\'s news?', 'label': 1},
    {'text': 'Would you please search today\'s news?', 'label': 1},
    {'text': 'What are the latest news?', 'label': 0}
]
# Create a dataframe from the list of dictionaries
df_examples = pd.DataFrame(dict_examples)
display(df_examples)
new_dataset_records = Dataset.from_pandas(df_examples)
print(new_dataset_records)

,text,label
0,Are you a news reporter?,0
1,Can you search the web for news?,0
2,"Hey there, search the web for news?",1
3,Can you search today's news?,1
4,Would you please search today's news?,1
5,What are the latest news?,0


Dataset({
    features: ['text', 'label'],
    num_rows: 6
})


Pushing to hub:

In [29]:
dataset_name = "reddgr/rq-request-question-prompts"
# Instantiate a labeling_widget object with the label map
manual_labeling_widget = labeling_widget.LabelingWidget(label_map)
# Push to Hugging Face hub directly by passing the dataframe with new examples to the update_dataset method
manual_labeling_widget.update_dataset(
    dataset_name=dataset_name,
    split_name="train", # Choose either test or train split
    hf_token=hf_token_write,
    new_dataset_records=new_dataset_records # The dataset we just created manually, without using the widget
)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Successfully pushed 6 records to reddgr/rq-request-question-prompts train split.


Check update:

In [30]:
dataset_new = load_dataset(dataset_name)
print(f"records in Train split: {len(dataset_new['train'])}\n...")
display(dataset_new['train'].to_pandas().tail(3))
print(f"records in Test split: {len(dataset_new['test'])}\n...")
display(dataset_new['test'].to_pandas().tail(3))

Generating train split:   0%|          | 0/118 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/182 [00:00<?, ? examples/s]

records in Train split: 118
...


,text,label
115,Can you search today's news?,1
116,Would you please search today's news?,1
117,What are the latest news?,0


records in Test split: 182
...


,text,label
179,Would you write me a poem?,1
180,Can you write me a poem?,1
181,Write me a poem,1
